In [1]:
import cv2
import mediapipe as mp
import pyautogui
import tkinter as tk
from tkinter import ttk, messagebox
from threading import Thread
from PIL import Image, ImageTk

# Variables for gesture control
x1 = y1 = x2 = y2 = 0
prev_dist = 0  # Previous distance to check for state change
threshold = 50  # Distance threshold to decide volume up or down
change_delay = 5  # Delay in frames between volume changes
frame_count = 0  # Frame counter
webcam = None
is_running = False
play_pause_state = False  # False means paused, True means playing

# Mediapipe hands setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
drawing_utils = mp.solutions.drawing_utils

# Theme variables
current_theme = "dark"  # Default theme is dark

# Background image setup
background_image_path = None
bg_image = None
bg_photo = None

def start_gesture_recognition():
    global webcam, is_running, frame_count, prev_dist, play_pause_state
    if is_running:
        messagebox.showinfo("Info", "Gesture recognition is already running!")
        return

    is_running = True
    status_label.config(text="Status: Running", foreground="light green")
    webcam = cv2.VideoCapture(0)

    def process_frames():
        global frame_count, prev_dist, play_pause_state
        while is_running:
            _, image = webcam.read()
            image = cv2.flip(image, 1)
            frame_height, frame_width, _ = image.shape
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            output = hands.process(rgb_image)

            if output.multi_hand_landmarks and output.multi_handedness:
                for hand_landmarks, hand_label in zip(output.multi_hand_landmarks, output.multi_handedness):
                    drawing_utils.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    landmarks = hand_landmarks.landmark
                    hand_type = hand_label.classification[0].label  # 'Left' or 'Right'

                    # Initialize visibility flags
                    index_finger_up = False
                    middle_finger_up = False

                    # Right Hand for Play/Pause
                    if hand_type == 'Right':
                        for id, landmark in enumerate(landmarks):
                            x = int(landmark.x * frame_width)
                            y = int(landmark.y * frame_height)
                            if id == 8:  # Index finger tip
                                cv2.circle(img=image, center=(x, y), radius=8, color=(0, 255, 255), thickness=3)
                                # Check if index finger is up
                                if landmarks[8].y < landmarks[7].y:  # If the tip is above the pip joint
                                    index_finger_up = True

                            if id == 12:  # Middle finger tip
                                cv2.circle(img=image, center=(x, y), radius=8, color=(255, 0, 0), thickness=3)
                                # Check if middle finger is up
                                if landmarks[12].y < landmarks[11].y:  # If the tip is above the pip joint
                                    middle_finger_up = True

                        # Play/Pause Gesture Detection
                        if index_finger_up and not middle_finger_up:  # Only index finger up (Pause)
                            if play_pause_state:  # If currently playing
                                pyautogui.press("playpause")  # Trigger pause
                                play_pause_state = False
                                frame_count = 0  # Reset frame count after action

                        elif index_finger_up and middle_finger_up:  # Index and middle fingers up (Play)
                            if not play_pause_state:  # If currently paused
                                pyautogui.press("playpause")  # Trigger play
                                play_pause_state = True
                                frame_count = 0  # Reset frame count after action

                    # Left Hand for Volume Control
                    elif hand_type == 'Left':
                        for id, landmark in enumerate(landmarks):
                            x = int(landmark.x * frame_width)
                            y = int(landmark.y * frame_height)
                            if id == 4:  # Thumb tip
                                cv2.circle(img=image, center=(x, y), radius=8, color=(0, 0, 255), thickness=3)
                                x1 = x
                                y1 = y

                            if id == 8:  # Index finger tip
                                cv2.circle(img=image, center=(x, y), radius=8, color=(0, 255, 255), thickness=3)
                                x2 = x
                                y2 = y

                        # Volume control by thumb and index finger distance
                        if len(landmarks) > 0:
                            # Calculate distance between thumb and index finger tips
                            dist = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5 // 4
                            cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

                            frame_count += 1
                            if frame_count > change_delay:
                                if dist > threshold and dist > prev_dist:
                                    pyautogui.press("volumeup")
                                    frame_count = 0  # Reset frame count after action
                                elif dist < threshold and dist < prev_dist:
                                    pyautogui.press("volumedown")
                                    frame_count = 0  # Reset frame count after action
                                prev_dist = dist  # Update previous distance

            cv2.imshow("Hand Gesture Volume and Playback Control", image)
            key = cv2.waitKey(10)
            if key == 27:  # ESC key to exit
                stop_gesture_recognition()

        webcam.release()
        cv2.destroyAllWindows()

    # Run frame processing in a separate thread
    Thread(target=process_frames).start()

def stop_gesture_recognition():
    global is_running
    if is_running:
        is_running = False
        status_label.config(text="Status: Stopped", foreground="red")
    else:
        messagebox.showinfo("Info", "Gesture recognition is not running!")

def open_settings():
    settings_window = tk.Toplevel(root)
    settings_window.title("Settings")
    settings_window.geometry("300x200")
    apply_theme(settings_window)  # Apply current theme

    # Threshold adjustment
    tk.Label(settings_window, text="Distance Threshold:", bg=settings_bg, fg=settings_fg).pack(pady=5)
    threshold_scale = tk.Scale(settings_window, from_=10, to=100, orient="horizontal", command=update_threshold,
                               bg=settings_bg, fg=settings_fg)
    threshold_scale.set(threshold)
    threshold_scale.pack(pady=5)

    # Delay adjustment
    tk.Label(settings_window, text="Change Delay (frames):", bg=settings_bg, fg=settings_fg).pack(pady=5)
    delay_scale = tk.Scale(settings_window, from_=1, to=10, orient="horizontal", command=update_delay, bg=settings_bg,
                           fg=settings_fg)
    delay_scale.set(change_delay)
    delay_scale.pack(pady=5)

def update_threshold(val):
    global threshold
    threshold = int(val)

def update_delay(val):
    global change_delay
    change_delay = int(val)

def toggle_theme():
    global current_theme
    if current_theme == "dark":
        current_theme = "light"
    else:
        current_theme = "dark"
    apply_theme(root)

def apply_theme(window):
    global settings_bg, settings_fg
    if current_theme == "dark":
        window.configure(bg="#1e1e1e")  # Dark background
        settings_bg = "#1e1e1e"
        settings_fg = "white"
        status_label.config(background="#1e1e1e", foreground="light green")
        start_button.config(style="Dark.TButton")
        stop_button.config(style="Dark.TButton")
        settings_button.config(style="Dark.TButton")
        theme_button.config(style="Dark.TButton")
        exit_button.config(style="Dark.TButton")
        theme_button.config(text="Switch to Light Mode")
    else:
        window.configure(bg="white")  # Light background
        settings_bg = "white"
        settings_fg = "black"
        status_label.config(background="white", foreground="green")
        start_button.config(style="Light.TButton")
        stop_button.config(style="Light.TButton")
        settings_button.config(style="Light.TButton")
        theme_button.config(style="Light.TButton")
        exit_button.config(style="Light.TButton")
        theme_button.config(text="Switch to Dark Mode")

def set_background_image():
    global background_image_path, bg_image, bg_photo
    file_path = r"C:\images\pexels-johnpet-1603891.jpg"  # Fixed path for the background image
    if file_path:
        background_image_path = file_path
        try:
            bg_image = Image.open(background_image_path)
            bg_image = bg_image.resize((400, 500), Image.Resampling.LANCZOS)
            bg_photo = ImageTk.PhotoImage(bg_image)
            bg_label.config(image=bg_photo)  # Update the background image
        except FileNotFoundError:
            messagebox.showerror("Error", "File not found. Please check the path and try again.")

def exit_application():
    if is_running:
        stop_gesture_recognition()
    root.destroy()

# GUI setup code
root = tk.Tk()
root.title("Hand Gesture Volume and Playback Control")
root.geometry("400x500")

# Background setup
bg_label = tk.Label(root)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)
set_background_image()

# Status Label
status_label = tk.Label(root, text="Status: Stopped", bg="#1e1e1e", fg="red")
status_label.pack(pady=10)

# Button styles
style = ttk.Style()
style.configure("Dark.TButton", background="#333", foreground="white", font=("Helvetica", 12, "bold"))
style.configure("Light.TButton", background="white", foreground="black", font=("Helvetica", 12, "bold"))

# Start, Stop, Settings, and Theme buttons
start_button = ttk.Button(root, text="Start Gesture Recognition", command=start_gesture_recognition, style="Dark.TButton")
start_button.pack(pady=5)

stop_button = ttk.Button(root, text="Stop Gesture Recognition", command=stop_gesture_recognition, style="Dark.TButton")
stop_button.pack(pady=5)

settings_button = ttk.Button(root, text="Settings", command=open_settings, style="Dark.TButton")
settings_button.pack(pady=5)

theme_button = ttk.Button(root, text="Switch to Light Mode", command=toggle_theme, style="Dark.TButton")
theme_button.pack(pady=5)

exit_button = ttk.Button(root, text="Exit", command=exit_application, style="Dark.TButton")
exit_button.pack(pady=5)

# Apply the initial theme
apply_theme(root)

# Start the main loop
root.mainloop()

C:\Users\ASUS\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
